# Great Expectations and YData Synthetic: An data expectation framework for a synthetic data engine

This notebook is a step-by-step tutorial on how you can inetgrate the great expectations framework while working on a ydata-synthetic project.
Specifically we will
1. Setup the project structure by initializing a Data Context
2. Download/Extract the real data set which we use to create synthetic data
3. Configure a Data Source to connect our data
4. Create an Expectation Suite using the built-in Great Expectations profiler
5. Transform the real data for modeling
6. Train the synethsizers and create the model file
7. Sample synthetic data from synthesizer
8. Inverse transform the data to obtain the original format
9. Create a new checkpoint to validate the synthetic data against the real data
10. Evaluate the synthetic data using Data Docs 

But before we get started, ensure that you have installed both ydata-synthetic and great_expectations after creating a virtual environment.

!pip intsall ydata-synthetic great_expetctaions

## Step 1: Setup the project structure through a Data Context

In Great Expectations, your Data Context manages the project configuration. There are multiple ways to create the Data Context, however the simplest one is by using the CLI that comes along when you install the great_expectations package.

Note: For this tutorial we'll use the newest V3 (Batch request) API so youd see the flag ```--v3-api``` in most of the commands.

Open your terminal and navigate to the project directory and type in the following:

```great_expectations --v3-api init```

Press enter to complete the creation of the Data Context and that's about it.

If you're curious about the modified project structure, here's an excerpt from the GE documentations:
- great_expectations.yml contains the main configuration of your deployment.
- The expectations/ directory stores all your Expectations as JSON files. If you want to store them somewhere else, you can change that later.
- The plugins/ directory holds code for any custom plugins you develop as part of your deployment.
- The uncommitted/ directory contains files that shouldn’t live in version control. It has a .gitignore configured to exclude all its contents from version control. The main contents of the directory are:
    - uncommitted/config_variables.yml, which holds sensitive information, such as database credentials and other secrets.
    - uncommitted/documentation, which contains Data Docs generated from Expectations, Validation Results, and other metadata.
    - uncommitted/validations, which holds Validation Results generated by Great Expectations.

## Step 2: Download/Extract the real data set which we use to create synthetic data

To move forward, we pick an use-case example of "The Credit Card Fraud Dataset - Synthesizing the Minority Class" where we aim to synthesize the minority class of the credit card fraud dataset that has a high imbalance.Further a practical exercise is presented to showcase the usage of the YData Synthetic library along with
GANs to synthesize tabular data. For the purpose of this exercise, dataset of credit card fraud from Kaggle is used, that can be found here:
https://www.kaggle.com/mlg-ulb/creditcardfraud

Since we're interested only on the fraud class data points of the dataset, let's filter them and write it to teh data directory.

In [1]:
# importing all required libraries

import os
import pandas as pd
import sklearn.cluster as cluster

from ydata_synthetic.synthesizers.regular import VanilllaGAN
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
from ydata_synthetic.preprocessing.regular.credit_fraud import *
from ydata_synthetic.postprocessing.regular.inverse_preprocesser import inverse_transform

model = VanilllaGAN

In [2]:
# Read the original data
data = pd.read_csv('./data/creditcard.csv')

#Filter the minority class
train_data = data.loc[ data['Class']==1 ].copy()

# Inspect the shape of the data
train_data.shape

(492, 31)

In [3]:
# Write to the data folder
train_data.to_csv('./data/creditcard_fraud.csv', index=False)

## Stpe 3: Configure a Data Source to connect our data

In Great Expectations, Datasources simplify connections, by managing configuration and providing a consistent, cross-platform API for referencing data.

Let’s configure our first Datasource: a connection to the data directory we’ve provided in the repo. Instead, this could even be a data base connection, and more.

```great_expectations --v3-api datasource new```

You would be presented with different options, select:
- 1. Files on a filesystem (for processing with Pandas or Spark) 

and for processing the data select


-  1. Pandas

and finally enter the directory as: data (where we have our real data)

Once you've entered the details an jupyter notebook will open up. This is just the way Great Expectations has given templated codes, which helps us create expectations with a few code changes.

Let's change the Datasource name to something more specific.

Edit the second code cell as follows:
```datasource_name = "data__dir"```

Then execute all cells in the notebook in order to save the new Datasource. If successful, the last cell will print a list of all Datasources, including the one you just created.

## Step 4: Create an Expectation Suite using the built-in Great Expectations profiler

An expectation is nothing but a falsifiable, verifiable statement about data. Expectations provide a language to talk about data characteristics and data quality - humans to humans, humans to machines and machines to machines.

The idea here is that we assume that the real data has the ideal quality of the data we want synthesized, so we use the real data to create a set of expectations which we can later use to evaluate our synthetic data on.

The CLI will help create our first Expectation Suite. Suites are simply collections of Expectations. We can use the built-in profiler to automatically create an Expectation Suite called `creditcard.quality`

Type the following into your terminal:
```great_expectations --v3-api suite new```

You would be presented with few options on the terminal. Choose 3 to create the Expectation Suite Automatically, using a profiler, and 1 to profile the  real fraud dataset creditcard_fraud.csv which we saved in step 2.

This step might be a bit confusing in the beginning, but stay with us. Again anotehr jupyter notebook would be opened with boilerplate code for creating new expectation suite. The code is pretty standard, however please not on the second cell, all columns are added to the list of ignored columns. In our example we want to validate every single column, hence remove (or comment) out the columns from the ignored_columns list.

Other than that, go ahead and execute the entire notebook. This will create an expectation suite against the real creditcard fraud dataset.

## Stpe 5: Transform the real data for modeling

Now that we have created the expectation suite, we shift our focus back to creating the synthetic data. We follow the standard process of transforming the data first.

In [5]:
# Extract list of columns
data_cols = list(data.columns[ data.columns != 'Class' ])

# Before training the GAN do not forget to apply the required data transformations
# To ease here we've applied a PowerTransformation - make data distribution more Gaussian-like.
_, data, preprocessor = transformations(data)

# For the purpose of this example we will only synthesize the minority class
# train_data contains 492 rows which had 'Class' value as 1 (which were very few)
train_data = data.loc[ data['Class']==1 ].copy()

print("Dataset info: Number of records - {} Number of variables - {}".format(train_data.shape[0], train_data.shape[1]))

# We define a K-means clustering method using sklearn, and declare that
# we want 2 clusters. We then apply this algorithm (fit_predict) to our train_data
# We essentially get an array of 492 rows ('labels') having values either 0 or 1 for the 2 clustered classes.
algorithm = cluster.KMeans
args, kwds = (), {'n_clusters':2, 'random_state':0}
labels = algorithm(*args, **kwds).fit_predict(train_data[ data_cols ])

# Get the count of both classes
print( pd.DataFrame( [ [np.sum(labels==i)] for i in np.unique(labels) ], columns=['count'], index=np.unique(labels) ) )

# Assign the k-means clustered classes' labels to the a seperate copy of train data 'fraud_w_classes'
fraud_w_classes = train_data.copy()
fraud_w_classes['Class'] = labels

Dataset info: Number of records - 492 Number of variables - 31
   count
0    375
1    117


In [6]:
train_data.shape

(492, 31)

# Step 6: Train the synethsizers and create the model

Below you can try to train your own generators using the available GANs architectures. You can train it either with labels (created using KMeans) or with no labels at all. 

Remember that for this exercise in particular we've decided to synthesize only the minority class from the Credit Fraud dataset.

In [7]:
# Define the GAN and training parameters
noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
epochs = 200+1
learning_rate = 5e-4
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'

train_sample = fraud_w_classes.copy().reset_index(drop=True)
print("train_sample.columns:")
print(train_sample.columns)

# There's only 1 class, so essentially rename the 'Class' to 'Class_1',
# which tells weather a sample data is of class 1 or not.
train_sample = pd.get_dummies(train_sample, columns=['Class'], prefix='Class', drop_first=True)

# 'Class_1' label
label_cols = [ i for i in train_sample.columns if 'Class' in i ]

# All columns except 'Class_1'
data_cols = [ i for i in train_sample.columns if i not in label_cols ]

# Scale down the data, and rename it to 'train_no_label'
train_sample[ data_cols ] = train_sample[ data_cols ] / 10 # scale to random noise size, one less thing to learn
train_no_label = train_sample[ data_cols ]

train_sample.columns:
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [8]:
#Setting the GAN model parameters and the training step parameters
gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           n_cols=train_sample.shape[1],
                           layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             sample_interval=log_step)

In [9]:
# Training the GAN model chosen: Vanilla GAN, CGAN, DCGAN, etc.
synthesizer = model(gan_args)
synthesizer.train(train_sample, train_args)

  1%|▊                                                                                 | 2/201 [00:00<01:16,  2.59it/s]

0 [D loss: 0.636076, acc.: 50.39%] [G loss: 0.666174]
generated_data
1 [D loss: 0.740944, acc.: 49.22%] [G loss: 0.543567]


  2%|█▋                                                                                | 4/201 [00:01<00:42,  4.66it/s]

2 [D loss: 0.721365, acc.: 49.22%] [G loss: 0.680754]
3 [D loss: 0.632399, acc.: 80.47%] [G loss: 0.947135]


  3%|██▍                                                                               | 6/201 [00:01<00:33,  5.79it/s]

...

196 [D loss: 0.528041, acc.: 75.00%] [G loss: 1.426011]
197 [D loss: 0.498662, acc.: 77.73%] [G loss: 1.432637]


100%|███████████████████████████████████████████████████████████████████████████████▌| 200/201 [00:28<00:00,  7.36it/s]

198 [D loss: 0.496180, acc.: 76.95%] [G loss: 1.309457]
199 [D loss: 0.548137, acc.: 74.61%] [G loss: 1.389564]


100%|████████████████████████████████████████████████████████████████████████████████| 201/201 [00:28<00:00,  7.09it/s]

200 [D loss: 0.498236, acc.: 77.34%] [G loss: 1.381582]
generated_data


In [10]:
# Generator description
synthesizer.generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(128, 32)]               0         
_________________________________________________________________
dense (Dense)                (128, 128)                4224      
_________________________________________________________________
dense_1 (Dense)              (128, 256)                33024     
_________________________________________________________________
dense_2 (Dense)              (128, 512)                131584    
_________________________________________________________________
dense_3 (Dense)              (128, 31)                 15903     
Total params: 184,735
Trainable params: 184,735
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Discriminator description
synthesizer.discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(128, 31)]               0         
_________________________________________________________________
dense_4 (Dense)              (128, 512)                16384     
_________________________________________________________________
dropout (Dropout)            (128, 512)                0         
_________________________________________________________________
dense_5 (Dense)              (128, 256)                131328    
_________________________________________________________________
dropout_1 (Dropout)          (128, 256)                0         
_________________________________________________________________
dense_6 (Dense)              (128, 128)                32896     
_________________________________________________________________
dense_7 (Dense)              (128, 1)                  129 

In [12]:
# You can easily save the trained generator and loaded it afterwards
if not os.path.exists("./saved/gan"):
    os.makedirs("./saved/gan")
synthesizer.save(path="./saved/gan/generator_fraud.pkl")

In [13]:
models = {'GAN': ['GAN', False, synthesizer.generator]}

## Step 7: Sample synthetic data from the Synthesizer

In [14]:
# use the same shape as the real data

synthetic_fraud = synthesizer.sample(492)

Synthetic data generation: 100%|████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 190.49it/s]


In [15]:
synthetic_fraud.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,-0.028322,-0.056973,0.075122,-0.077015,0.176086,0.000401,-0.006622,-0.054043,0.023125,-0.138701,...,0.033368,-0.051680,0.035562,-0.060543,0.052678,0.003060,0.028167,0.003305,-0.097905,-0.024802
1,0.003791,-0.060411,0.116220,-0.123338,0.207262,-0.004066,-0.038257,-0.122285,0.059356,-0.159980,...,0.055218,-0.054451,0.027974,-0.054083,0.038001,0.035491,0.046303,0.008580,-0.114265,-0.016638
2,-0.084681,0.027821,-0.008397,-0.090557,0.092042,-0.026930,0.042580,0.026009,-0.003500,-0.001856,...,0.042665,-0.100598,0.006049,-0.049122,0.036208,-0.045092,0.004077,-0.017562,-0.063882,-0.046820
3,0.024048,-0.055298,-0.004408,-0.118931,0.139287,-0.070903,-0.009979,-0.057477,0.043256,-0.128425,...,0.045004,-0.013676,0.112154,-0.037354,0.024974,0.044594,0.063829,-0.083527,0.011412,-0.039752
4,0.090478,-0.014989,0.027560,-0.088822,0.147422,0.037545,0.017765,-0.013605,-0.002967,-0.097704,...,0.051536,-0.086814,0.045123,-0.031647,-0.003462,-0.010022,-0.021095,-0.008208,-0.047879,-0.011579


However we notice that the generated synthetic data is still on the transformed form and needs to be inverse-transformed to the original format.

## Step 8: Inverse transform the data to obtain the original format

In [16]:
synthetic_data = inverse_transform(synthetic_fraud,preprocessor )

C:\Users\arunnt\Miniconda3\envs\synth\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  "X does not have valid feature names, but"


In [17]:
synthetic_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.038542,-0.032114,0.064474,-0.079785,0.179731,0.000191,0.007486,-0.054481,0.011590,-0.134371,...,-0.008922,0.035842,-0.052928,0.033230,-0.061040,0.049955,0.003665,0.027846,0.008150,-0.095918
1,0.070174,-0.035555,0.105647,-0.126073,0.210845,-0.004276,-0.024071,-0.122712,0.047731,-0.155687,...,-0.007819,0.057671,-0.055698,0.025641,-0.054581,0.035273,0.036095,0.045982,0.013416,-0.112287
2,-0.017405,0.052484,-0.019017,-0.093318,0.095807,-0.027140,0.056479,0.025569,-0.014931,0.002582,...,0.005489,0.045131,-0.101832,0.003719,-0.049621,0.033479,-0.044488,0.003756,-0.012690,-0.061880
3,0.090039,-0.030438,-0.015035,-0.121670,0.142993,-0.071112,0.004139,-0.057914,0.031664,-0.124079,...,-0.026704,0.047467,-0.014928,0.109859,-0.037854,0.022243,0.045197,0.063509,-0.078634,0.013425
4,0.154732,0.009834,0.016898,-0.091585,0.151116,0.037336,0.031782,-0.014045,-0.014400,-0.093318,...,-0.008395,0.053993,-0.088053,0.042792,-0.032147,-0.006194,-0.009417,-0.021416,-0.003347,-0.045873


In [18]:
#Add back the class column
synthetic_data['Class']= 1

#Write the data for profiling
synthetic_data.to_csv('./data/creditcard_fraud_synthetic.csv', index=False)

## Step 9: Create a new checkpoint to validate the synthetic data against the real data

For the normal usage of Great Expectations, the best way to validate data is with a Checkpoint. Checkpoints bundle Batches of data with corresponding Expectation Suites for validation.

From the terminal, run the following command
```great_expectations --v3-api checkpoint new my_new_checkpoint```

This will again open a Jupyter Notebook that will allow you to complete the configuration of our Checkpoint. Edit the data_asset_name to reference the data we want to validate as "creditcard_fraud_synthetic.csv" (the file we wrote in step 8). Ensure that the expectation_suite_name is identical to what we created in step 4.

Once done, go ahead and execute all the cells in the notebook.

## Step 10: Evaluate the synthetic data using Data Docs

If you've following alone, you would have created the new checkpoint to validate the synthetic data. The last final step is to uncomment the final cell of the checkpoint notebook and execute it.

This will open up a HTML page titled Data Docs. We can inspect the Data Docs, for the most recent check point and see that the expectation has failed. By clicking on the checkpoint run, we get a detailed report of which expectations failed from which columns.

Based on this input we can do either of these actions:

- Go back to our synthesizer and tweak the parameters, optimize it to get better synthetic data
- Go back to the expectation suite and edit few expectations that is not important (maybe for certain columns). Yes - the expectations are customizable and here's how you can do it https://docs.greatexpectations.io/docs/guides/expectations/creating_custom_expectations/how_to_create_custom_expectations

# Conclusion

In this tutorial we have successfully demonstrated the use of ydata-synthetic alongside great expectations. A 10 step guide was presented starting from configuring a data context to evaluating the synthesized data using Data Docs. We believe an integration of the these two libraries can help data sciencetist unlock the power of synthetic data with data quality.